# Técnicas de *Tiling* para el Procesamiento de Imágenes SAR en el Contexto de Aprendizaje Automático


Este codigo combina las complejas imagenes VH,VV en imagenes *tiles* con el proposito de entrenar y testear un algoritmo de detección y clasificacion. 

## Introducción

El procesamiento avanzado de imágenes de radar de apertura sintética (SAR), especialmente en aplicaciones de aprendizaje automático, requiere técnicas eficientes para manejar grandes volúmenes de datos. El tiling o segmentación en mosaicos, es crucial para la gestión de imágenes grandes, facilitando el procesamiento paralelo y reduciendo la carga computacional.

## Metodología

El código analizado combina imágenes complejas SLC en polarizaciones VH y VV para formar imágenes en tiles, destinadas al entrenamiento de algoritmos de detección y clasificación. Se describen los pasos desde la lectura de las imágenes hasta su segmentación en mosaicos, destacando la importancia del cálculo de magnitud y fase para cada imagen procesada.

### Preparación y Ajuste de Imágenes

Se utiliza *rasterio* para ajustar dimensiones de las imágenes mediante padding, asegurando que cada imagen sea divisible exactamente en tiles de dimensiones especificadas, considerando la superposición entre ellos.

### Cálculo de Magnitud y Fase

Se calculan la magnitud y fase de las imágenes VH y VV, almacenando los resultados en un array de NumPy con cuatro canales, lo cual es esencial para las subsiguientes etapas de clasificación y detección.

### Segmentación en *Tiling*

Este proceso implica combinar las imágenes ajustadas en un array de cuatro canales y segmentar este array en tiles cuadrados, que se almacenan individualmente, facilitando su uso en entrenamientos de modelos de aprendizaje profundo.

## Estructura de Archivos Generada

Posteriormente, se crea una estructura de archivos donde cada tile contiene un array de NumPy con los cuatro canales calculados, adecuado para el entrenamiento de modelos:


```bash


[TilePath]/[scene_id]/swath1/0_0.npy    | This is the chip data (complex numpy array), 
                                        | the filename is tx_ty.npy, where tx is the tile index in x axis,
                                        | and ty is the tile index in y axis
                                        | Each .npy has 4 channels, vh_mag, vh_phase, vv_mag, vv_phase

[TilePath]/img_file_info.csv            | This file saved the file info of the scenes used
```

In [40]:
import numpy as np
from pathlib import Path
import numpy as np
import rasterio
import matplotlib.pyplot as plt

# Make Matplotlib plots appear inline
%matplotlib inline

## <combineVHVV:>

### Lectura y Ajuste de Imágenes:

```python
with rasterio.open(vhFN) as src:
    vhImg = src.read(1)  # Lee la primera banda
```

    
Las imágenes SAR se leen utilizando la librería rasterio, que facilita el manejo de archivos GeoTIFF. Este paso inicial implica la lectura del primer canal de la imagen, generalmente representando una polarización específica del radar. Posteriormente, la imagen se ajusta en tamaño mediante un proceso de padding. Esto consiste en agregar filas y columnas de ceros a la imagen original para asegurar que las dimensiones de la imagen sean múltiplos del tamaño de los tiles, menos el solapamiento entre ellos.

### Cálculo de Magnitud y Fase:

```python
padImg = np.pad(vhImg, pad_width=((0, padH), (0, padW)), mode="constant", constant_values=0)
vhAbs = np.abs(padImg)
vhPhase = np.angle(padImg)
```

Para cada píxel de las imágenes VH y VV, se calcula la magnitud y la fase del número complejo correspondiente. Este paso es esencial porque las magnitudes y fases son características clave que se utilizan en la detección y clasificación en muchas aplicaciones de procesamiento de imágenes SAR. La función np.pad se utiliza para ajustar las dimensiones de la imagen, y las funciones np.abs y np.angle son utilizadas para calcular la magnitud y la fase, respectivamente.


### Combinación de Canales y Creación de Tiles:

out = np.stack([vhAbs, vhPhase, vvAbs, vvPhase], axis=0)

Los valores calculados de magnitud y fase para cada polarización se combinan en un único arreglo de cuatro canales. Este arreglo multidimensional se crea apilando los canales de magnitud y fase de las imágenes VH y VV. Posteriormente, este arreglo se segmenta en tiles, que son pequeños segmentos de la imagen de dimensiones predefinidas. Estos tiles se almacenan individualmente, lo que facilita su posterior procesamiento o uso en entrenamiento de modelos.


In [4]:


def combineVHVV(vhFN: Path, vvFN: Path, tileSize: int, tileOverlap: int) -> np.ndarray:
    """Combine VH and VV images to produce a 4-channel numpy array with absolute and phase values.
    
    Args:
    vhFN (Path): Path to the VH GeoTIFF file.
    vvFN (Path): Path to the VV GeoTIFF file.
    tileSize (int): Size of the tile for processing.
    tileOverlap (int): Overlap size of the tiles.
    
    Returns:
    np.ndarray: 4-channel numpy array containing the absolute and phase of VH and VV.
    """
    # Read VH image using rasterio
    with rasterio.open(vhFN) as src:
        vhImg = src.read(1)  # Read the first band

    imgH, imgW = vhImg.shape
    s = tileSize - tileOverlap
    newH = int(np.ceil((imgH - tileOverlap) / s) * s) + tileOverlap
    newW = int(np.ceil((imgW - tileOverlap) / s) * s) + tileOverlap
    padH = newH - imgH
    padW = newW - imgW

    # Padding the image
    padImg = np.pad(vhImg, pad_width=((0, padH), (0, padW)), mode="constant", constant_values=0)
    vhAbs = np.abs(padImg)
    vhPhase = np.angle(padImg)

    # Read VV image using rasterio
    with rasterio.open(vvFN) as src:
        vvImg = src.read(1)  # Read the first band

    padImg = np.pad(vvImg, pad_width=((0, padH), (0, padW)), mode="constant", constant_values=0)
    vvAbs = np.abs(padImg)
    vvPhase = np.angle(padImg)

    # Combine into a 4-channel array
    out = np.stack([vhAbs, vhPhase, vvAbs, vvPhase], axis=0)
    return out


## <chopAndSaveTiles:>


```python
for tx in range(numTileX):
    for ty in range(numTileY):
        x1 = tx * s
        y1 = ty * s
        x2 = x1 + tileSize
        y2 = y1 + tileSize
        FN = Path(outDir, f"{tx}_{ty}.npy")
        with open(str(FN), "wb") as f:
            np.save(f, combined[:, y1:y2, x1:x2])
```

Este fragmento de código se encarga de dividir el arreglo combinado en tiles y guardar cada tile como un archivo .npy en el directorio especificado. Se calculan las coordenadas de cada tile basándose en el tamaño del tile y el solapamiento, y cada tile se guarda de manera que pueda ser accesible para procesos de aprendizaje automático posteriores.

In [5]:
def chopAndSaveTiles(
        combined: np.ndarray, outDir: Path , tileSize: int, tileOverlap: int
    ):
    """Chop the combined image (4 x H x W) into tiles
    """
    s = tileSize - tileOverlap
    imgH = combined.shape[1]
    imgW = combined.shape[2]
    numTileX = int( imgW / s )
    numTileY = int( imgH / s )
    if not outDir.exists():
        outDir.mkdir(parents = True, exist_ok = True)

    for tx in range( numTileX ):
        for ty in range( numTileY ):
            x1 = tx * s
            y1 = ty * s
            x2 = x1 + tileSize
            y2 = y1 + tileSize
            FN = Path(outDir, f"{tx}_{ty}.npy")
            with open(str(FN), "wb") as f:
                np.save( f, combined[ :, y1:y2, x1:x2 ] )


In [34]:
foldDf = pd.read_csv("fold.csv")

sceneList = foldDf['scene_id'].values
swathList = [1, 2, 3]

data_path =  "../../data/"

xView3_SLC_GRD_correspondences = pd.read_csv("../../data/SARFish" + "/labels/xView3_SLC_GRD_correspondences.csv")

xView3_SLC_GRD_correspondences = xView3_SLC_GRD_correspondences[
    xView3_SLC_GRD_correspondences["scene_id"].isin(sceneList)
]

SARFish_root_directory = Path(data_path)

product_type = "SLC"

tileSize = 1024

tileOverlap = 64


tilePath = Path("extracted_slc_chip/")

if not tilePath.exists():
    tilePath.mkdir(parents = True, exist_ok = True)

measurement_directory = Path(SARFish_root_directory, product_type, row["DATA_PARTITION"],  f"{row[f'{product_type}_product_identifier']}.SAFE", "measurement",  )


measurement_directory

## Guardado de Tiles

Cada tile se guarda en formato .npy. Esto facilita la carga y manipulación de los datos durante la fase de entrenamiento de modelos de machine learning.

In [35]:
for index, row in xView3_SLC_GRD_correspondences.iterrows():
    for swath_index in [1, 2, 3]:
        outDir = Path(tilePath, row[f"{product_type}_product_identifier"], f"swath{swath_index}")
        if outDir.exists():
            continue

        measurement_directory = Path(
            SARFish_root_directory, product_type, row["DATA_PARTITION"], 
            f"{row[f'{product_type}_product_identifier']}.SAFE", "measurement", 
        )
        vh_FN = Path(measurement_directory, row[f"SLC_swath_{swath_index}_vh"])
        vv_FN = Path(measurement_directory, row[f"SLC_swath_{swath_index}_vv"])

        # Check if files exist
        if not vh_FN.exists() or not vv_FN.exists():
            print(f"Error: Files do not exist at {vh_FN} and/or {vv_FN}")
            continue

        print(f"tiling {row[f'{product_type}_product_identifier']}, swath {swath_index}, VH at {vh_FN}, VV at {vv_FN}")
        combined = combineVHVV(vh_FN, vv_FN, tileSize, tileOverlap)
        chopAndSaveTiles(combined, outDir, tileSize, tileOverlap)


tiling S1B_IW_SLC__1SDV_20200803T075720_20200803T075748_022756_02B2FF_E5D2, swath 1, VH at ../../data/SLC/validation/S1B_IW_SLC__1SDV_20200803T075720_20200803T075748_022756_02B2FF_E5D2.SAFE/measurement/s1b-iw1-slc-vh-20200803t075720-20200803t075748-022756-02b2ff-001_SARFish.tiff, VV at ../../data/SLC/validation/S1B_IW_SLC__1SDV_20200803T075720_20200803T075748_022756_02B2FF_E5D2.SAFE/measurement/s1b-iw1-slc-vv-20200803t075720-20200803t075748-022756-02b2ff-004_SARFish.tiff
tiling S1B_IW_SLC__1SDV_20200803T075720_20200803T075748_022756_02B2FF_E5D2, swath 2, VH at ../../data/SLC/validation/S1B_IW_SLC__1SDV_20200803T075720_20200803T075748_022756_02B2FF_E5D2.SAFE/measurement/s1b-iw2-slc-vh-20200803t075721-20200803t075746-022756-02b2ff-002_SARFish.tiff, VV at ../../data/SLC/validation/S1B_IW_SLC__1SDV_20200803T075720_20200803T075748_022756_02B2FF_E5D2.SAFE/measurement/s1b-iw2-slc-vv-20200803t075721-20200803t075746-022756-02b2ff-005_SARFish.tiff
tiling S1B_IW_SLC__1SDV_20200803T075720_20200803